In [1]:
import requests
import json
import csv
import datetime
from time import sleep

def write_csv(filename,new_line):
    with open(filename, "a" ) as fout:
        writer=csv.writer(fout)
        writer.writerow((new_line))
        
fut = "XBTZ17"
spot = "XBTUSD"
pos = 20000
timeframe = 58  # в секундах
cycle = 10
filename = "arb_data.txt"

k=0

In [2]:
def download(fut, spot, pos, timeframe, filename):    
    
    # Запрос стакана по фьючерсу
    fut_url="https://www.bitmex.com/api/v1/orderBook?symbol="+fut+"&depth=25"
    fut_html=requests.get(fut_url).text
    fut_data=json.loads(fut_html)

    # Запрос стакана по споту
    spot_url="https://www.bitmex.com/api/v1/orderBook?symbol="+spot+"&depth=25"
    spot_html=requests.get(spot_url).text
    spot_data=json.loads(spot_html)

    # Получаем данные о дате поставки фьючерса и количество дней до экспирации
    fut_setl_url="https://www.bitmex.com/api/v1/instrument?symbol="+fut+"&count=2&reverse=true"
    fut_setl_html=requests.get(fut_setl_url).text
    fut_setl_data=json.loads(fut_setl_html)
    
    YYYY = int(fut_setl_data[0]['settle'][:4])
    MM = int(fut_setl_data[0]['settle'][5:7])
    DD = int(fut_setl_data[0]['settle'][8:10])

    Settle_Date = datetime.date(YYYY,MM,DD)
    Curr_Date = datetime.datetime.utcnow().date()
    Curr_Datetime = datetime.datetime.strftime(datetime.datetime.utcnow(), "%Y.%m.%d %H:%M:%S")
    Days_to_settle = Settle_Date - Curr_Date
    Days_to_settle = int(str(Days_to_settle)[:2])
    Days_for_funding = str(Days_to_settle*3)
        
            
    # Получаем взвешенные по объему позиции цены
    pos_test = pos+pos*0.1
        
    q=0
    w=[]
    e=[]
    for i in fut_data:           
        q=q+i["askSize"]    
        if q>pos_test:
            w.append([i["askSize"], i["askPrice"]])
        if q<pos_test:
            e.append([i["askSize"], i["askPrice"]])      
    r=0
    t=0
    for j in e:
        r=j[0]*j[1]+r
        t=j[0]+t      
    r=w[0][1]*(pos_test-t)+r
    fut_ask=round(r/pos_test)   


    q=0
    w=[]
    e=[]
    for i in fut_data:           
        q=q+i["bidSize"]    
        if q>pos_test:
            w.append([i["bidSize"], i["bidPrice"]])
        if q<pos_test:
            e.append([i["bidSize"], i["bidPrice"]])      
    r=0
    t=0
    for j in e:
        r=j[0]*j[1]+r
        t=j[0]+t      
    r=w[0][1]*(pos_test-t)+r
    fut_bid=round(r/pos_test)
    

    q=0
    w=[]
    e=[]
    for i in spot_data:           
        q=q+i["askSize"]    
        if q>pos_test:
            w.append([i["askSize"], i["askPrice"]])
        if q<pos_test:
            e.append([i["askSize"], i["askPrice"]])      
    r=0
    t=0
    for j in e:
        r=j[0]*j[1]+r
        t=j[0]+t      
    r=w[0][1]*(pos_test-t)+r
    spot_ask=round(r/pos_test)


    q=0
    w=[]
    e=[]
    for i in spot_data:           
        q=q+i["bidSize"]    
        if q>pos_test:
            w.append([i["bidSize"], i["bidPrice"]])
        if q<pos_test:
            e.append([i["bidSize"], i["bidPrice"]])      
    r=0
    t=0
    for j in e:
        r=j[0]*j[1]+r
        t=j[0]+t      
    r=w[0][1]*(pos_test-t)+r
    spot_bid=round(r/pos_test)
        
    # Расчет фондирования по свопу        
    #funding_url="https://www.bitmex.com/api/v1/funding?symbol="+spot+"&count="+Days_for_funding+"&reverse=true"
    #funding_html=requests.get(funding_url).text
    #funding_data=json.loads(funding_html)
        
    #funding_rate = []
    #for i in funding_data:
    #    funding_rate.append(i["fundingRate"])
        
    #funding_rate_settle = round(sum(funding_rate), 6)
    
    # Расчет спредов (Спред = Фьючерс - СПОТ, значит при "+" спреде - продаем спред (продаем фьючерс, покупаем СПОТ)
    # при "-" - покупаем.)
    #Simple_Spread_Buy = fut_ask-spot_bid
    #Simple_Spread_Sell = fut_bid-spot_ask
            
    fut_commis = 0.000750
    spot_commis = 0.000750
            
    Spread_Buy = round(fut_ask-spot_bid-(fut_ask*fut_commis)-(spot_bid*spot_commis),2)
    Spread_Sell = round(fut_bid-spot_ask-(fut_bid*fut_commis)-(spot_ask*spot_commis),2)
    
    Spread_Buy_pct = round((((fut_ask-(fut_ask*fut_commis))/(spot_bid+(spot_bid*spot_commis)))
                            **(1/float(Days_to_settle))-1)*36500,4)
    Spread_Sell_pct =round((((fut_bid+(fut_bid*fut_commis))/(spot_ask-(spot_ask*spot_commis)))
                            **(1/float(Days_to_settle))-1)*36500,4)     
            
    new_line=[Curr_Datetime, Days_to_settle, "fut_ask", fut_ask, "fut_bid", fut_bid, "spot_ask", spot_ask,
              "spot_bid", spot_bid, "Sp_Buy", Spread_Buy, "Sp_Sell", Spread_Sell, "Sp_Buy_pct", Spread_Buy_pct,
              "Sp_Sell_pct", Spread_Sell_pct]    
        
    return write_csv(filename, new_line)               

In [3]:
while k < cycle:
    try:        
        download(fut, spot, pos, timeframe, filename)
        
        sleep(timeframe)
        k=k+1
        print k, "success"
        
    except:
        print k+1, "fail"
        pass


1 success
2 success
3 success
4 success
5 success
6 success
7 success
8 success
9 success
10 success
